In [9]:
import random
import cx_Oracle
import pandas as pd
import datetime 

In [10]:
cx_Oracle.init_oracle_client(lib_dir=r"C:\OracleDBproject\instantclient_21_7")  # 여기서 각자 라이브러리 위치 설정해줘야함
connection = cx_Oracle.connect(user='admin', password='Semorang1234', dsn='semodb_high')
# 본인이 접속할 오라클 클라우드 DB 사용자이름, 비밀번호, dsn을 넣어준다.
print(connection)

ProgrammingError: Oracle Client library has already been initialized

In [10]:
def execute_sql(query):
    cursor = connection.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    # return len(rows)

In [3]:
execute_sql("select * from RANKING_SNAPSHOT")

NameError: name 'execute_sql' is not defined

In [18]:
def desc(table):
    cur = connection.cursor()
    cur.execute("select * from "+table)
    print(" (name, type, display_size, internal_size, precision, scale, null_ok)")
    for column in cur.description:
        print(column)

In [19]:
desc("ranking_snapshot")

 (name, type, display_size, internal_size, precision, scale, null_ok)
('SNAPSHOT_ID', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 11, 11, None, None, 0)
('THING_RANK_ID', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 11, 11, None, None, 1)
('USER_ID', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 11, 11, None, None, 1)
('CONTENT', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 3000, 3000, None, None, 0)
('TAKEN_DATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1)
('UPDATE_DATE', <cx_Oracle.DbType DB_TYPE_DATE>, 23, None, None, None, 1)
('LIKE_COUNT', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 10, 0, 1)
('DISLIKE_COUNT', <cx_Oracle.DbType DB_TYPE_NUMBER>, 11, None, 10, 0, 1)


In [11]:
## 게시물 내용 만들기
lyric = """어쩐지 오늘 알람이 안 울려
아무거나 막 입었고 지각이야
아래로 몸을 낮추고 숨을 꾹 참아봤지만
아이쿠 들켰나 봐
내 맘이 방심할 때마다 불쑥 나타난 뒤
헤엄치듯 멀어지는 너
또 물보라를 일으켜
Da-da-da-da, da-da-da-da-da-da
Da-da-da-da 또 물보라를 일으켜
Da-da-da-da, da-da-da-da-da-da
Da-da-da-da 또 물보라를 일으켜
이 파도 위 난 온종일
서핑을 하는 느낌 아찔한 feeling
난 솔직히 푹 빠졌지
하지만 이건 secret
또 물보라를 일으켜
갑자기 웃으며 얼굴을 불쑥 다가온 뒤
원을 갈라 하트를 그려
또 물보라를 일으켜
Da-da-da-da, da-da-da-da-da-da
Da-da-da-da 또 물보라를 일으켜
Da-da-da-da, da-da-da-da-da-da
Da-da-da-da 또 물보라를 일으켜
Da-da-da-da, da-da-da-da-da-da
Da-da-da-da 또 물보라를 일으켜
Da-da, da-da-da-da-da-da
Da-da-da-da 또 물보라를 일으켜
단팥을 넣었으면 단팥빵
크림을 넣었으면 크림빵
겨울엔 따끈따끈 호빵
아침엔 똥글똥글 모닝빵
가루 가루 밀가루 반죽은
치대고 치대면 부풀어 오르고
오랜 시간 손맛은
우리를 배신하지 않는다
빵 먹고 빵빵 뱃속이 빵빵
빵 먹고 빵빵 든든해 빵빵
딸기잼도 샐러드도
발라버려 어울려
빵 먹다 뻑뻑하면 우유를 한잔
빵 먹다 너무달아 커피를 한잔
꼭꼭 씹어 먹다보면
살살 녹아버린다
빵빵빵빵 빵 빵빵
빵빵빵빵 빵 빵빵
빵빵빵빵공공칠 빵
안녕하세요
제 이름은 빵빵빵 입니다
50줄 체워야되서 쓰는데 뭘 써야될까요
50줄 체웠나 확인해볼까요?"""
lyric = lyric.split('\n')
lyric = lyric[:50]

In [12]:

def random_snapshot_data(snapshot_id,user_id):
        rank_snap_tup = []

        rank_snap_id = snapshot_id
        user = user_id
        content = lyric[random.randint(0,49)]

        taken_year = str(random.randint(2019,2020))
        taken_month = str(random.randint(1,12))
        taken_day = str(random.randint(1,28))
        taken_date = taken_year + '-' + taken_month + '-' + taken_day
        taken_date = datetime.datetime.strptime(taken_date, '%Y-%m-%d')

        update_year = str(random.randint(2021,2022))
        update_month = str(random.randint(1,12))
        update_day = str(random.randint(1,28))
        update_date = update_year + '-' + update_month + '-' + update_day
        update_date = datetime.datetime.strptime(update_date, '%Y-%m-%d')

        like = random.randint(0,10*100)
        dislike = random.randint(0,10*100)

        return ((rank_snap_id, user, content, taken_date, update_date, like, dislike))

In [13]:
# n 명의 유저에 대한 thingrank를 가져오고 그중에서 랜덤하게 선택해서 데이터 생성
def make_thingrank_user(n):
    # user ID 다 가져오기
    cursor = connection.cursor()
    cursor.execute("select U.USER_ID from USERS U")
    user_rows = cursor.fetchall()
    # print(len(rows))

    count = 0
    while count < n:
        # user 1명 랜덤 선택
        user = user_rows[random.randint(0,len(user_rows)-1)][0]
        cursor = connection.cursor()
        # user 1명에 대한 thing_rank 전부 가져오기        
        cursor.execute("select TR.THING_RANK_ID from USERS U ,THINGRANK TR WHERE U.USER_ID = TR.USER_ID AND U.USER_ID = :TEMP_USER_ID",TEMP_USER_ID = user)
        tr_rows = cursor.fetchall()
        # print(tr_rows)
        
        # 해당 user에 대한 thingrank가 있다면 snapshot 생성
        if(len(tr_rows)>0):
            # snapshot id 1개 생성
            snapshot_id = "snapshot_" + str(count+1)
            # RS insert문 생성
            cursor.execute("INSERT INTO RANKING_SNAPSHOT VALUES (:1,:2,:3,:4,:5,:6,:7)", random_snapshot_data(snapshot_id,user))
            # S_include_TR insert문 i개 생성
            i = random.randint(1,len(tr_rows))
            for j in range(0,i):
                cursor.execute("INSERT INTO S_INCLUDE_TR VALUES (:SNAPSHOT_ID,:THING_RANK_ID)",SNAPSHOT_ID=snapshot_id,THING_RANK_ID=tr_rows[j][0])
            count +=1
        else:
            pass


In [14]:
make_thingrank_user(99)

In [15]:
connection.commit()